In [1]:
%matplotlib inline
import cv2
import matplotlib.pyplot as plt
import logging

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

In [2]:
import mxnet as mx
import numpy as np

from sklearn import svm
from sklearn import metrics
from sklearn import preprocessing
from imblearn.over_sampling import SMOTE
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

import os
import sys
import math
import time
import pickle

proj_dir = os.getcwd()
label_dir = os.path.join(proj_dir, 'emotiondetection/features_labels_lld/labels')
lld_dir = os.path.join(proj_dir, 'emotiondetection/features_labels_lld/lld')

pickle_train_y_list = 'pickle_train_y_list'
pickle_train_x_list = 'pickle_train_x_list'

pickle_test_y_list = 'pickle_test_y_list'
pickle_test_x_list = 'pickle_test_x_list'

In [3]:
train_y_list = pickle.load(open(pickle_train_y_list))
train_x_list = pickle.load(open(pickle_train_x_list))

test_y_list = pickle.load(open(pickle_test_y_list))
test_x_list = pickle.load(open(pickle_test_x_list))

In [4]:
def calc_score(test_y_list, predict_y_list):
    print "Resulting Accuracy (AVG), Recall (AVG), F-Score (AVG):"
    print 'macro   : ', metrics.precision_recall_fscore_support(test_y_list, predict_y_list, average='macro')
    print 'weighted: ', metrics.precision_recall_fscore_support(test_y_list, predict_y_list, average='weighted')

In [5]:
# z-score normalization
# scaler = preprocessing.StandardScaler().fit(train_x_list)
# train_x_list = scaler.transform(train_x_list)
# test_x_list = scaler.transform(test_x_list)

In [6]:
# PCA whiten
# pca = PCA(n_components='mle', svd_solver='full', whiten=True)
# x_pca = pca.fit(train_x_list)
# train_x_list = pca.transform(train_x_list)
# test_x_list = pca.transform(test_x_list)

In [7]:
# sklearn l1/l2 normalization
# x_normalizer = preprocessing.Normalizer(norm='l2').fit(train_x_list)
# train_x_list = x_normalizer.transform(train_x_list)
# test_x_list = x_normalizer.transform(test_x_list)

In [8]:
# # SMOTE up-sampling training data
# sm = SMOTE(kind='regular')
# train_x_list, train_y_list = sm.fit_sample(train_x_list, train_y_list)
# train_x_list, train_y_list = sm.fit_sample(train_x_list, train_y_list)
# train_x_list, train_y_list = sm.fit_sample(train_x_list, train_y_list)
# train_x_list, train_y_list = sm.fit_sample(train_x_list, train_y_list)
# train_x_list, train_y_list = sm.fit_sample(train_x_list, train_y_list)

# # split training data to 5 different classes
# train_x_lists = [[] for i in range(5)]
# for i in xrange(len(train_y_list)):
#     train_x_lists[int(train_y_list[i])].append(train_x_list[i])
# for i in xrange(5):
#     print 'number of class', i, ':', len(train_x_lists[i])

In [9]:
print train_y_list.shape, train_x_list.shape
print test_y_list.shape, test_x_list.shape

(9959,) (9959, 384)
(8257,) (8257, 384)


In [10]:
print train_x_list[0][:10]
print test_x_list[0][:10]

[  5.66733100e-03   3.11773100e-04   5.35555800e-03   5.30000000e+01
   9.60000000e+01   1.30774500e-03   4.18643100e-06   1.09633100e-03
   1.80239600e-06   1.34818000e-03]
[  1.23016500e-02   9.11361400e-05   1.22105100e-02   5.20000000e+01
   2.40000000e+01   1.31725500e-03   3.23349300e-07   1.29866300e-03
   6.89166500e-06   2.62522000e-03]


In [11]:
X = train_x_list
y = train_y_list

In [12]:
# Linear Discriminant Analysis with solver="svd"
print 'start:', time.strftime( '%Y-%m-%d %H:%M:%S', time.localtime( time.time() ) )
lda = LinearDiscriminantAnalysis(solver="svd", store_covariance=True)
predict_y_list = lda.fit(X, y).predict(test_x_list)
calc_score(test_y_list, predict_y_list)

print 'end:', time.strftime( '%Y-%m-%d %H:%M:%S', time.localtime( time.time() ) )

start: 2016-12-27 13:38:07
Resulting Accuracy (AVG), Recall (AVG), F-Score (AVG):
macro   :  (0.35936656041590076, 0.3375721378731863, 0.34056468342636742, None)
weighted:  (0.58814106817086764, 0.63558193048322631, 0.60588664292089445, None)
end: 2016-12-27 13:38:07


/home/slhome/cc001/anaconda3/envs/python2/lib/python2.7/site-packages/sklearn/discriminant_analysis.py:387: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


In [13]:
# Linear Discriminant Analysis with solver='lsqr'
print 'start:', time.strftime( '%Y-%m-%d %H:%M:%S', time.localtime( time.time() ) )
lda = LinearDiscriminantAnalysis(solver='lsqr', shrinkage='auto')
predict_y_list = lda.fit(X, y).predict(test_x_list)
calc_score(test_y_list, predict_y_list)

print 'end:', time.strftime( '%Y-%m-%d %H:%M:%S', time.localtime( time.time() ) )

start: 2016-12-27 13:38:07
Resulting Accuracy (AVG), Recall (AVG), F-Score (AVG):
macro   :  (0.37080283667407332, 0.33423314477594146, 0.33960041194534063, None)
weighted:  (0.58996760752335464, 0.64430180452949015, 0.60782833325293195, None)
end: 2016-12-27 13:38:07


In [14]:
# Quadratic Discriminant Analysis
print 'start:', time.strftime( '%Y-%m-%d %H:%M:%S', time.localtime( time.time() ) )

qda = QuadraticDiscriminantAnalysis(store_covariances=True)
predict_y_list = qda.fit(X, y).predict(test_x_list)
calc_score(test_y_list, predict_y_list)

print 'end:', time.strftime( '%Y-%m-%d %H:%M:%S', time.localtime( time.time() ) )

start: 2016-12-27 13:38:08


/home/slhome/cc001/anaconda3/envs/python2/lib/python2.7/site-packages/sklearn/discriminant_analysis.py:695: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


Resulting Accuracy (AVG), Recall (AVG), F-Score (AVG):
macro   :  (0.48686372745064377, 0.28079546552893037, 0.22556947756713636, None)
weighted:  (0.60372976351401275, 0.39723870655201649, 0.43285330988382714, None)
end: 2016-12-27 13:38:08


In [15]:
# SMOTE up-sampling training data
sm = SMOTE(kind='regular')
train_x_list_smote, train_y_list_smote = sm.fit_sample(train_x_list, train_y_list)
train_x_list_smote, train_y_list_smote = sm.fit_sample(train_x_list_smote, train_y_list_smote)
train_x_list_smote, train_y_list_smote = sm.fit_sample(train_x_list_smote, train_y_list_smote)
train_x_list_smote, train_y_list_smote = sm.fit_sample(train_x_list_smote, train_y_list_smote)
train_x_list_smote, train_y_list_smote = sm.fit_sample(train_x_list_smote, train_y_list_smote)

# split training data to 5 different classes
train_x_lists = [[] for i in range(5)]
for i in xrange(len(train_y_list_smote)):
    train_x_lists[int(train_y_list_smote[i])].append(train_x_list_smote[i])
for i in xrange(5):
    print 'number of class', i, ':', len(train_x_lists[i])

No handlers could be found for logger "imblearn.base"


number of class 0 : 5590
number of class 1 : 5590
number of class 2 : 5590
number of class 3 : 5590
number of class 4 : 5590


In [16]:
# Linear Discriminant Analysis with solver="svd"
print 'start:', time.strftime( '%Y-%m-%d %H:%M:%S', time.localtime( time.time() ) )
lda = LinearDiscriminantAnalysis(solver="svd", store_covariance=True)
predict_y_list = lda.fit(train_x_list_smote, train_y_list_smote).predict(test_x_list)
calc_score(test_y_list, predict_y_list)

print 'end:', time.strftime( '%Y-%m-%d %H:%M:%S', time.localtime( time.time() ) )

start: 2016-12-27 13:38:10
Resulting Accuracy (AVG), Recall (AVG), F-Score (AVG):
macro   :  (0.31983691553048404, 0.40551613869639913, 0.30412227330388331, None)
weighted:  (0.61049818218883789, 0.38863994186750633, 0.44353416744668239, None)
end: 2016-12-27 13:38:11


In [17]:
# Linear Discriminant Analysis with solver='lsqr'
print 'start:', time.strftime( '%Y-%m-%d %H:%M:%S', time.localtime( time.time() ) )
lda = LinearDiscriminantAnalysis(solver='lsqr', shrinkage='auto')
predict_y_list = lda.fit(train_x_list_smote, train_y_list_smote).predict(test_x_list)
calc_score(test_y_list, predict_y_list)

print 'end:', time.strftime( '%Y-%m-%d %H:%M:%S', time.localtime( time.time() ) )

start: 2016-12-27 13:38:11
Resulting Accuracy (AVG), Recall (AVG), F-Score (AVG):
macro   :  (0.32304495367600838, 0.40444138581083555, 0.31056995147312744, None)
weighted:  (0.61101146702576581, 0.39881312825481408, 0.45261388596287161, None)
end: 2016-12-27 13:38:12


In [18]:
# Quadratic Discriminant Analysis
print 'start:', time.strftime( '%Y-%m-%d %H:%M:%S', time.localtime( time.time() ) )

qda = QuadraticDiscriminantAnalysis(store_covariances=True)
predict_y_list = qda.fit(train_x_list_smote, train_y_list_smote).predict(test_x_list)
calc_score(test_y_list, predict_y_list)

print 'end:', time.strftime( '%Y-%m-%d %H:%M:%S', time.localtime( time.time() ) )

start: 2016-12-27 13:38:12
Resulting Accuracy (AVG), Recall (AVG), F-Score (AVG):
macro   :  (0.3253596161924831, 0.25961671959830029, 0.23915926462076978, None)
weighted:  (0.5183577372675745, 0.51301925638851886, 0.50489772078555573, None)
end: 2016-12-27 13:38:13


/home/slhome/cc001/anaconda3/envs/python2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [19]:
print predict_y_list[:100]

[ 1.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  1.  2.  1.  2.
  2.  1.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  0.  1.  2.  2.
  1.  0.  2.  0.  2.  2.  0.  1.  2.  1.  1.  1.  1.  2.  0.  2.  2.  2.
  1.  2.  2.  2.  2.  2.  2.  1.  2.  1.  2.  1.  1.  2.  2.  2.  2.  1.
  1.  2.  1.  2.  1.  2.  1.  1.  1.  1.  1.  1.  2.  1.  1.  1.  1.  1.
  2.  2.  1.  2.  2.  2.  2.  1.  1.  2.]


In [20]:
print test_y_list[:100]

[2 2 2 2 2 4 2 2 2 2 2 2 2 2 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1
 1 2 2 2 2 2 2 2 1 1 2 1 1 2 2 2 0 1 2 2 2 2 2 2 2 2 2 2 1 1 2 2 2 2 2 2 2
 1 1 1 2 1 1 1 1 1 2 2 1 1 1 2 1 1 2 2 2 2 2 1 1 1 2]
